In [19]:
import numpy as np
from bayes_opt import BayesianOptimization

data = np.load('../game_results/game_results_humans.npz')
xyz = data['xyz']
params = data['params']
meta = data['meta']

# Ground truth model in numpy
def gmm_model(params, x, y):
    # params: (N, 25), x/y: (N, 20)
    g = params[:24].reshape(6, 4)
    w, mx, my, s = g[:, 0], g[:,1], g[:, 2], g[:, 3]
    expo = -((x[...,None] - mx[None])**2 + (y[...,None] - my[None])**2) / s[None]
    res = np.sum(w[None] * np.exp(expo), axis=-1)
    
    return res * params[24]


# for i in range(len(params)):
#     assert np.allclose(gmm_model(params[i], xyz[i,:,0], xyz[i,:,1]), xyz[i,:,2])

In [20]:
# import numpy as np
# from skopt.space import Integer
# import skopt.space
# from skopt import gp_minimize
# from skopt.utils import use_named_args

# # grid
# x = np.linspace(0,1,100,endpoint=False)
# grid = np.meshgrid(x,x)
# search_space = [skopt.space.Integer(0, 99, name='x'), skopt.space.Integer(0, 99, name='y')]
# #objective function
# def f_objective(point):
#     x,y=point
#     return objective[x,y]

# for person in xyz[:2]:
#     for i,trial in enumerate(person):
#         x0=[[int(x0[0]*100),int(x0[1]*100)] for x0 in xyz[i][:4]]

#         for k,point in enumerate(trial):
#             objective=-gmm_model(params[k], grid[0], grid[1])
#             res=gp_minimize(f_objective, search_space, n_calls=20, 
#                             x0=[[int(xy[0]*100),int(xy[1]*100)] for xy in trial[:len(x0)]],acq_func='EI')
#             print(res['x_iters'])
#             x0.append(res['x_iters'][-1])

In [21]:
# import numpy as np
# x = np.linspace(0,1,100,endpoint=False)
# grid = np.meshgrid(x,x)
# def f_objective(point):
#     x,y=point
#     return objective[x,y]


# from skopt.space import Integer
# import skopt.space
# from skopt import gp_minimize
# from skopt.utils import use_named_args
# search_space = [skopt.space.Integer(0, 99, name='x'), skopt.space.Integer(0, 99, name='y')]
# for i in range(2,3):
#     objective=-gmm_model(params[i], grid[0], grid[1])
#     res=gp_minimize(f_objective, search_space, n_calls=20,n_initial_points=0, 
#                     x0=[[int(x0[0]*100),int(x0[1]*100)] for x0 in xyz[i][:4]],acq_func='EI',
#                     acq_optimizer='auto')


# print(len(res['models']))
# print(len(res['x_iters']))
# print(len(res['func_vals']))
# res

In [22]:
# print(res['models'][0])

In [23]:
import sys
from pathlib import Path
import numpy as np

project_root = Path.cwd().parent
sys.path.append(str(project_root))

from BayesianOptimization.BO_functions import probability_of_improvement, expected_improvement, GP_UCB_original, squared_exponential_kernel, fit_predictive_GP, optimize_GP_hyperparams

In [25]:
np.random.seed(32)
X = np.linspace(0,1,100,endpoint=False)
xx, yy = np.meshgrid(X,X)
Xtest = np.column_stack([xx.ravel(), yy.ravel()])

def f_objective(point):
    x,y=point
    x = min(99,int(x*100))
    y = min(99, int(y*100))
    return objective[x,y]

prior_mean = 0 
prior_std = 5
eps = 0.5
    
results = []

for i, person in enumerate(xyz[:2]):
    # First 4 points
    print(f"Running person {i}")
    x0 = [[x0[0],x0[1]] for x0 in person[:4]]
    objective = -gmm_model(params[i], xx, yy)            
    X_sample =  x0
    y_sample = [f_objective((xy[0],xy[1])) for xy in x0]
    current_best = np.max(y_sample)

    for k, point in enumerate(person[4:19]):
        lengthscale, output_variance, noise_variance = optimize_GP_hyperparams(X_sample, y_sample, 500, 5e-3, prior_mean, prior_std)
        mu, covariance = fit_predictive_GP(X_sample, y_sample, Xtest, lengthscale, output_variance, noise_variance)
        std = np.sqrt(np.diag(covariance))

        acquisition_values_pi = probability_of_improvement(current_best,  mu.flatten(), std, eps)
        acquisition_values_ei = expected_improvement(current_best,  mu.flatten(), std, eps)
        acquisition_values_gp_ucb = GP_UCB_original(mu.flatten(), std, eps)

        # BO-optimal points for each acquisition
        idx_PI  = np.argmax(acquisition_values_pi)
        idx_EI  = np.argmax(acquisition_values_ei)
        idx_UCB = np.argmax(acquisition_values_gp_ucb)

        xt_PI  = Xtest[idx_PI]
        xt_EI  = Xtest[idx_EI]
        xt_UCB = Xtest[idx_UCB]

        results.append({
            "person": i,
            "step": k,
            "human_xy": list(point[:2]),
            "human_z": point[2],

            "bo_PI_xy": xt_PI,
            "bo_EI_xy": xt_EI,
            "bo_UCB_xy": xt_UCB,

            "PI_max": acquisition_values_pi[idx_PI],
            "EI_max": acquisition_values_ei[idx_EI],
            "UCB_max": acquisition_values_gp_ucb[idx_UCB],

            "current_best": current_best,

            "all_PI_values": acquisition_values_pi,
            "all_EI_values": acquisition_values_ei,
            "all_UCB_values": acquisition_values_gp_ucb 
        })

        X_sample.append(list(point[:2]))
        y_sample.append(point[2])
        current_best = np.max(y_sample)


Running person 0
Running person 1


In [30]:
results

[{'person': 0,
  'step': 0,
  'human_xy': [np.float64(0.128), np.float64(0.7021666564941407)],
  'human_z': np.float64(0.28064131881437193),
  'bo_PI_xy': array([0.43, 0.5 ]),
  'bo_EI_xy': array([0.43, 0.5 ]),
  'bo_UCB_xy': array([0., 0.]),
  'PI_max': np.float64(2.906069770935083e-153),
  'EI_max': np.float64(2.44488108705966e-156),
  'UCB_max': np.float64(0.026786540509649664),
  'current_best': np.float64(-1.4713530501277127e-07),
  'all_PI_values': array([0., 0., 0., ..., 0., 0., 0.], shape=(10000,)),
  'all_EI_values': array([0., 0., 0., ..., 0., 0., 0.], shape=(10000,)),
  'all_UCB_values': array([ 0.02678654,  0.02463876,  0.02249166, ..., -0.16833203,
         -0.16912278, -0.16985673], shape=(10000,))},
 {'person': 0,
  'step': 1,
  'human_xy': [np.float64(0.238), np.float64(0.40416665649414063)],
  'human_z': np.float64(0.3776535316477387),
  'bo_PI_xy': array([0., 0.]),
  'bo_EI_xy': array([0., 0.]),
  'bo_UCB_xy': array([0., 0.]),
  'PI_max': np.float64(0.0),
  'EI_max': 

In [ ]:
# import json

# # Save to json black magic

# # Convert results so that everything is JSON-serializable
# def make_json_serializable(results):
#     serializable = []
#     for entry in results:
#         new_entry = {}
#         for k, v in entry.items():
#             if isinstance(v, np.ndarray):
#                 new_entry[k] = v.tolist()  # convert arrays to lists
#             elif isinstance(v, (np.floating, np.float32, np.float64)):
#                 new_entry[k] = float(v)    # convert numpy floats to Python floats
#             else:
#                 new_entry[k] = v
#         serializable.append(new_entry)
#     return serializable

# serializable_results = make_json_serializable(results)

# # Save to JSON
# with open("results.json", "w") as f:
#     json.dump(serializable_results, f, indent=2)

In [ ]:
import json

# Load from json
with open("results.json", "r") as f:
    loaded_results = json.load(f)

# Convert acquisition values back to numpy arrays
for entry in loaded_results:
    entry["all_PI_values"] = np.array(entry["all_PI_values"])
    entry["all_EI_values"] = np.array(entry["all_EI_values"])
    entry["all_UCB_values"] = np.array(entry["all_UCB_values"])
    entry["bo_PI_xy"] = np.array(entry["bo_PI_xy"])
    entry["bo_EI_xy"] = np.array(entry["bo_EI_xy"])
    entry["bo_UCB_xy"] = np.array(entry["bo_UCB_xy"])

In [46]:
loaded_results

[{'person': 0,
  'step': 0,
  'human_xy': [0.128, 0.7021666564941407],
  'human_z': 0.28064131881437193,
  'bo_PI_xy': array([0.43, 0.5 ]),
  'bo_EI_xy': array([0.43, 0.5 ]),
  'bo_UCB_xy': array([0., 0.]),
  'PI_max': 2.906069770935083e-153,
  'EI_max': 2.44488108705966e-156,
  'UCB_max': 0.026786540509649664,
  'current_best': -1.4713530501277127e-07,
  'all_PI_values': array([0., 0., 0., ..., 0., 0., 0.], shape=(10000,)),
  'all_EI_values': array([0., 0., 0., ..., 0., 0., 0.], shape=(10000,)),
  'all_UCB_values': array([ 0.02678654,  0.02463876,  0.02249166, ..., -0.16833203,
         -0.16912278, -0.16985673], shape=(10000,))},
 {'person': 0,
  'step': 1,
  'human_xy': [0.238, 0.40416665649414063],
  'human_z': 0.3776535316477387,
  'bo_PI_xy': array([0., 0.]),
  'bo_EI_xy': array([0., 0.]),
  'bo_UCB_xy': array([0., 0.]),
  'PI_max': 0.0,
  'EI_max': 0.0,
  'UCB_max': 0.00015815150997261954,
  'current_best': 0.28064131881437193,
  'all_PI_values': array([0., 0., 0., ..., 0., 0., 